# Critical Zone Thermodynamics
## Understanding Energy Flow in Earth's Critical Zone

**Learning Objectives:**
- Understand thermodynamic principles governing Critical Zone processes
- Calculate energy balance components and their interactions
- Explore threshold behaviors and system transitions
- Visualize energy flux patterns across climate gradients

**Prerequisites:**
- Basic thermodynamics and energy concepts
- Python programming fundamentals
- Understanding of earth system processes

**Estimated Time:** 45 minutes

## 1. Environment Setup

In [ ]:
# Environment verification
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Check conda environment
if 'CONDA_DEFAULT_ENV' in os.environ:
    env_name = os.environ['CONDA_DEFAULT_ENV']
    if env_name != 'eemt-gis':
        print(f"⚠️  Current environment: {env_name}")
        print("💡 Recommended: conda activate eemt-gis")
    else:
        print(f"✅ Using correct environment: {env_name}")
else:
    print("⚠️  No conda environment detected")

print(f"🐍 Python version: {sys.version.split()[0]}")

In [ ]:
# Core scientific computing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats, optimize
import sympy as sp
from sympy import symbols, Eq, solve, latex, init_printing

# Interactive visualization
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from ipywidgets import interact, FloatSlider, Dropdown, VBox, HBox

# Configure display
init_printing(use_latex=True)
plt.style.use('seaborn-v0_8')
sns.set_palette("viridis")

# Set random seed for reproducibility
np.random.seed(42)

print("✅ Packages loaded successfully")

## 2. The Critical Zone: Earth's Energy Processing System

The **Critical Zone** is Earth's near-surface environment that extends from the vegetation canopy down to groundwater. This zone functions as an open thermodynamic system where energy and matter flow according to fundamental physical laws.

In [ ]:
# Create Critical Zone conceptual diagram
fig = go.Figure()

# Add atmospheric layer
fig.add_shape(
    type="rect", x0=0, y0=8, x1=10, y1=10,
    fillcolor="lightblue", opacity=0.3,
    line=dict(width=0)
)

# Add vegetation layer
fig.add_shape(
    type="rect", x0=0, y0=6, x1=10, y1=8,
    fillcolor="green", opacity=0.4,
    line=dict(width=0)
)

# Add soil layers
colors = ['#8B4513', '#CD853F', '#DEB887', '#F5DEB3']
for i, color in enumerate(colors):
    fig.add_shape(
        type="rect", x0=0, y0=6-2*i-2, x1=10, y1=6-2*i,
        fillcolor=color, opacity=0.6,
        line=dict(width=0)
    )

# Add groundwater
fig.add_shape(
    type="rect", x0=0, y0=-2, x1=10, y1=0,
    fillcolor="blue", opacity=0.3,
    line=dict(width=0)
)

# Add energy flux arrows
energy_flows = [
    dict(x=2, y=9, dx=0, dy=-1, color='yellow', name='Solar Radiation'),
    dict(x=8, y=7, dx=0, dy=1, color='red', name='Evapotranspiration'),
    dict(x=5, y=5, dx=0, dy=-1, color='orange', name='Precipitation Energy'),
    dict(x=1, y=3, dx=1, dy=0, color='brown', name='Chemical Weathering'),
    dict(x=7, y=1, dx=1, dy=0, color='gray', name='Physical Erosion')
]

for flow in energy_flows:
    fig.add_annotation(
        x=flow['x'], y=flow['y'],
        ax=flow['x'] + flow['dx'], ay=flow['y'] + flow['dy'],
        xref="x", yref="y", axref="x", ayref="y",
        arrowhead=2, arrowsize=1, arrowwidth=3,
        arrowcolor=flow['color']
    )

# Add labels
labels = [
    dict(x=1, y=9, text="Atmosphere", size=14),
    dict(x=1, y=7, text="Vegetation Canopy", size=12),
    dict(x=1, y=5, text="O Horizon (Organic)", size=10),
    dict(x=1, y=3, text="A Horizon (Topsoil)", size=10),
    dict(x=1, y=1, text="B Horizon (Subsoil)", size=10),
    dict(x=1, y=-1, text="C Horizon (Bedrock)", size=10),
    dict(x=1, y=-1.5, text="Groundwater", size=10)
]

for label in labels:
    fig.add_annotation(
        x=label['x'], y=label['y'], text=label['text'],
        showarrow=False, font=dict(size=label['size'], color='black')
    )

fig.update_layout(
    title="Critical Zone: Energy and Mass Transfer System",
    xaxis=dict(range=[0, 10], showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(range=[-2, 10], showgrid=False, zeroline=False, showticklabels=False),
    showlegend=False,
    width=800, height=600,
    annotations=[
        dict(x=5, y=9.5, text="CRITICAL ZONE BOUNDARY", 
             showarrow=False, font=dict(size=16, color='red'))
    ]
)

fig.show()

## 3. Thermodynamic Foundation

### 3.1 Open System Energy Balance

The Critical Zone operates as an **open thermodynamic system** following fundamental conservation laws:

In [ ]:
# Define symbolic variables for thermodynamic equations
U_cz, K, T, sigma, t = symbols('U_{CZ} K T sigma t', real=True, positive=True)
Q, W, S = symbols('Q W S', real=True)
T_0 = symbols('T_0', real=True, positive=True)

# First Law of Thermodynamics (Energy Conservation)
first_law = Eq(sp.diff(U_cz, t), K - T*sigma)

print("First Law of Thermodynamics for the Critical Zone:")
display(first_law)
print("\nWhere:")
print("• U_CZ = Critical Zone energy storage [J m⁻²]")
print("• K = Energy flux through the system [W m⁻²]")
print("• T = System temperature [K]")
print("• σ = Entropy production rate [W m⁻² K⁻¹]")

In [ ]:
# Energy balance components
E_ET, E_PPT, E_BIO, E_ELEV, E_GEO, E_other = symbols(
    'E_{ET} E_{PPT} E_{BIO} E_{ELEV} E_{GEO} E_{other}', 
    real=True, positive=True
)

# Total energy flux equation
total_energy = Eq(K, E_ET + E_PPT + E_BIO + E_ELEV + E_GEO + E_other)

print("Total Energy Flux Components:")
display(total_energy)

# EEMT focus: subsurface energy flux
EEMT = E_BIO + E_PPT
eemt_eq = Eq(symbols('EEMT'), EEMT)

print("\nEffective Energy and Mass Transfer (EEMT):")
display(eemt_eq)

### 3.2 Energy Component Magnitudes

Understanding the relative magnitudes of different energy fluxes helps prioritize which processes to focus on:

In [ ]:
# Typical energy flux magnitudes (MJ m⁻² yr⁻¹)
energy_components = {
    'E_ET (Evapotranspiration)': 10**5,
    'E_PPT (Precipitation)': 10**2, 
    'E_BIO (Primary Production)': 10**1,
    'E_ELEV (Physical Erosion)': 10**0,
    'E_GEO (Chemical Weathering)': 10**(-1)
}

# Create magnitude comparison plot
components = list(energy_components.keys())
magnitudes = list(energy_components.values())

fig = go.Figure()

# Add bars with logarithmic scale
fig.add_trace(go.Bar(
    x=components,
    y=np.log10(magnitudes),
    name='Energy Flux',
    marker_color=['red', 'blue', 'green', 'orange', 'purple'],
    text=[f'{mag:.0e}' for mag in magnitudes],
    textposition='auto'
))

# Highlight EEMT components
eemt_indices = [1, 2]  # E_PPT and E_BIO
fig.add_shape(
    type="rect",
    x0=-0.5, y0=-1.5, x1=2.5, y1=2.5,
    line=dict(color="red", width=3),
    fillcolor="rgba(255,0,0,0.1)"
)

fig.add_annotation(
    x=1, y=2.8,
    text="EEMT Focus Area",
    showarrow=True,
    arrowhead=2,
    arrowcolor="red",
    font=dict(size=14, color="red")
)

fig.update_layout(
    title="Energy Flux Component Magnitudes in the Critical Zone",
    xaxis_title="Energy Component",
    yaxis_title="Log₁₀(Energy Flux) [MJ m⁻² yr⁻¹]",
    showlegend=False,
    width=900, height=500
)

fig.show()

print("\n💡 Key Insight: EEMT focuses on subsurface energy fluxes (E_PPT + E_BIO)")
print("   These represent the energy effectively transferred to drive Critical Zone processes.")

## 4. EEMT Component Calculations

### 4.1 Biological Energy Component (E_BIO)

In [ ]:
# Biological energy calculation
NPP, h_BIO = symbols('NPP h_{BIO}', real=True, positive=True)

# E_BIO equation
e_bio_eq = Eq(E_BIO, NPP * h_BIO)

print("Biological Energy Component:")
display(e_bio_eq)
print("\nWhere:")
print("• NPP = Net Primary Production [kg m⁻² s⁻¹]")
print("• h_BIO = Specific biomass enthalpy (22 × 10⁶ J kg⁻¹)")

# Example calculation
def calculate_e_bio(npp_annual, h_bio=22e6):
    """
    Calculate biological energy component
    
    Parameters:
    npp_annual: Net primary production [kg/m²/yr]
    h_bio: Specific biomass enthalpy [J/kg]
    
    Returns:
    E_BIO in [W/m²] and [MJ/m²/yr]
    """
    # Convert annual NPP to flux [kg/m²/s]
    seconds_per_year = 365 * 24 * 3600
    npp_flux = npp_annual / seconds_per_year
    
    # Calculate E_BIO [W/m²]
    e_bio_flux = npp_flux * h_bio
    
    # Convert to annual energy [MJ/m²/yr]
    e_bio_annual = e_bio_flux * seconds_per_year / 1e6
    
    return e_bio_flux, e_bio_annual

# Example with typical NPP range
npp_range = np.logspace(-1, 0.5, 50)  # 0.1 to 3 kg/m²/yr
e_bio_values = [calculate_e_bio(npp) for npp in npp_range]
e_bio_annual = [val[1] for val in e_bio_values]

# Plot E_BIO vs NPP relationship
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=npp_range,
    y=e_bio_annual,
    mode='lines',
    name='E_BIO',
    line=dict(color='green', width=3)
))

# Add ecosystem type annotations
ecosystems = [
    {'name': 'Desert', 'npp': 0.1, 'color': 'brown'},
    {'name': 'Grassland', 'npp': 0.5, 'color': 'orange'},
    {'name': 'Temperate Forest', 'npp': 1.2, 'color': 'green'},
    {'name': 'Tropical Forest', 'npp': 2.5, 'color': 'darkgreen'}
]

for eco in ecosystems:
    e_bio_val = calculate_e_bio(eco['npp'])[1]
    fig.add_trace(go.Scatter(
        x=[eco['npp']], y=[e_bio_val],
        mode='markers',
        name=eco['name'],
        marker=dict(size=10, color=eco['color'])
    ))

fig.update_layout(
    title="Biological Energy Component (E_BIO) vs. Net Primary Production",
    xaxis_title="NPP [kg m⁻² yr⁻¹]",
    yaxis_title="E_BIO [MJ m⁻² yr⁻¹]",
    xaxis_type="log",
    width=800, height=500
)

fig.show()

### 4.2 Precipitation Energy Component (E_PPT)

In [ ]:
# Precipitation energy calculation
F, c_w, Delta_T = symbols('F c_w Delta_T', real=True, positive=True)

# E_PPT equation
e_ppt_eq = Eq(E_PPT, F * c_w * Delta_T)

print("Precipitation Energy Component:")
display(e_ppt_eq)
print("\nWhere:")
print("• F = Effective precipitation flux [kg m⁻² s⁻¹]")
print("• c_w = Specific heat of water (4.18 × 10³ J kg⁻¹ K⁻¹)")
print("• ΔT = Temperature above freezing [K]")

def calculate_e_ppt(precip_annual, temperature, c_w=4180):
    """
    Calculate precipitation energy component
    
    Parameters:
    precip_annual: Effective precipitation [mm/yr]
    temperature: Mean temperature [°C]
    c_w: Specific heat of water [J/kg/K]
    
    Returns:
    E_PPT in [W/m²] and [MJ/m²/yr]
    """
    # Convert precipitation to mass flux [kg/m²/s]
    seconds_per_year = 365 * 24 * 3600
    precip_flux = (precip_annual / 1000) / seconds_per_year  # mm to m, then to kg/m²/s
    
    # Temperature difference from freezing
    delta_t = max(0, temperature - 0)  # Above 0°C
    
    # Calculate E_PPT [W/m²]
    e_ppt_flux = precip_flux * c_w * delta_t
    
    # Convert to annual energy [MJ/m²/yr]
    e_ppt_annual = e_ppt_flux * seconds_per_year / 1e6
    
    return e_ppt_flux, e_ppt_annual

# Create interactive E_PPT calculator
@interact(
    precipitation=FloatSlider(min=100, max=2000, step=50, value=800, 
                             description='Precipitation (mm/yr)'),
    temperature=FloatSlider(min=-5, max=25, step=1, value=15,
                           description='Temperature (°C)')
)
def interactive_e_ppt(precipitation, temperature):
    e_ppt_flux, e_ppt_annual = calculate_e_ppt(precipitation, temperature)
    
    print(f"📊 E_PPT Calculation Results:")
    print(f"   Precipitation: {precipitation} mm/yr")
    print(f"   Temperature: {temperature}°C")
    print(f"   E_PPT: {e_ppt_flux:.3f} W/m²")
    print(f"   E_PPT: {e_ppt_annual:.1f} MJ/m²/yr")
    
    # Classification based on magnitude
    if e_ppt_annual < 50:
        classification = "Low energy (arid/cold)"
    elif e_ppt_annual < 200:
        classification = "Moderate energy (temperate)"
    else:
        classification = "High energy (warm/wet)"
    
    print(f"   Classification: {classification}")

## 5. EEMT Threshold Behavior

### 5.1 The 70 MJ/m²/yr Transition

Research has identified a critical threshold around 70 MJ/m²/yr where Critical Zone systems transition between different organizational states:

In [ ]:
def eemt_threshold_analysis():
    """
    Demonstrate EEMT threshold behavior
    """
    # Generate synthetic EEMT values
    eemt_values = np.logspace(0, 3, 100)  # 1 to 1000 MJ/m²/yr
    
    # Define threshold
    threshold = 70  # MJ/m²/yr
    
    # Calculate relative contributions of E_BIO and E_PPT
    # Below threshold: E_BIO dominates (water-limited)
    # Above threshold: E_PPT dominates (energy-limited)
    
    f_bio = np.where(eemt_values < threshold,
                     0.7 - 0.2 * (eemt_values / threshold),  # E_BIO dominant
                     0.5 * np.exp(-(eemt_values - threshold) / 100))  # E_BIO decreases
    
    f_ppt = 1 - f_bio
    
    # Create threshold visualization
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=['Energy Component Ratios', 'System Properties',
                       'Biomass Scaling', 'Soil Development'],
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    # Plot 1: Energy component ratios
    fig.add_trace(
        go.Scatter(x=eemt_values, y=f_bio, name='E_BIO fraction', 
                  line=dict(color='green', width=3)),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=eemt_values, y=f_ppt, name='E_PPT fraction',
                  line=dict(color='blue', width=3)),
        row=1, col=1
    )
    
    # Add threshold line
    fig.add_vline(x=threshold, line_dash="dash", line_color="red", 
                  annotation_text="Threshold: 70 MJ/m²/yr",
                  row=1, col=1)
    
    # Plot 2: System properties
    productivity = 0.032 * eemt_values**1.22  # Empirical relationship
    fig.add_trace(
        go.Scatter(x=eemt_values, y=productivity, name='Productivity',
                  line=dict(color='orange', width=2)),
        row=1, col=2
    )
    
    # Plot 3: Biomass scaling (power law)
    biomass = 0.032 * eemt_values**3.22  # From literature
    fig.add_trace(
        go.Scatter(x=eemt_values, y=np.log10(biomass), name='Log Biomass',
                  line=dict(color='darkgreen', width=2)),
        row=2, col=1
    )
    
    # Plot 4: Soil development
    soil_depth = 0.5 * (1 - np.exp(-eemt_values / 100))  # Exponential approach
    fig.add_trace(
        go.Scatter(x=eemt_values, y=soil_depth, name='Soil Depth',
                  line=dict(color='brown', width=2)),
        row=2, col=2
    )
    
    # Add threshold lines to all subplots
    for row in [1, 2]:
        for col in [1, 2]:
            fig.add_vline(x=threshold, line_dash="dash", line_color="red",
                         row=row, col=col)
    
    fig.update_layout(
        title="EEMT Threshold Behavior in Critical Zone Systems",
        width=1000, height=700
    )
    
    # Update x-axes to log scale
    fig.update_xaxes(type="log", title_text="EEMT [MJ m⁻² yr⁻¹]")
    
    # Update y-axes labels
    fig.update_yaxes(title_text="Energy Fraction", row=1, col=1)
    fig.update_yaxes(title_text="Productivity [kg/m²/yr]", row=1, col=2)
    fig.update_yaxes(title_text="Log₁₀(Biomass)", row=2, col=1)
    fig.update_yaxes(title_text="Soil Depth [m]", row=2, col=2)
    
    fig.show()
    
    # Print threshold interpretation
    print("🔄 EEMT Threshold Interpretation:")
    print(f"   Below {threshold} MJ/m²/yr: Water-limited systems")
    print("   • E_BIO dominates energy flux")
    print("   • Vegetation controls soil development")
    print("   • Examples: Arid/semiarid ecosystems")
    print()
    print(f"   Above {threshold} MJ/m²/yr: Energy-limited systems")
    print("   • E_PPT dominates energy flux")
    print("   • Climate controls weathering rates")
    print("   • Examples: Humid temperate/tropical ecosystems")

eemt_threshold_analysis()

## 6. Climate Gradient Analysis

### 6.1 EEMT Across Environmental Gradients

Let's explore how EEMT varies across different climate conditions:

In [ ]:
def create_climate_gradient_analysis():
    """
    Analyze EEMT patterns across climate gradients
    """
    # Define climate gradient
    n_points = 50
    
    # Temperature gradient (°C)
    temperature = np.linspace(-2, 28, n_points)
    
    # Precipitation gradient (mm/yr)
    precipitation = np.linspace(200, 2500, n_points)
    
    # Create meshgrid for 2D analysis
    T_grid, P_grid = np.meshgrid(temperature, precipitation)
    
    # Calculate EEMT components across the grid
    EEMT_grid = np.zeros_like(T_grid)
    E_BIO_grid = np.zeros_like(T_grid)
    E_PPT_grid = np.zeros_like(T_grid)
    
    for i in range(n_points):
        for j in range(n_points):
            temp = T_grid[i, j]
            precip = P_grid[i, j]
            
            # Simple NPP model (temperature and precipitation dependent)
            if temp > 0 and precip > 300:
                npp = min(2.0, 0.1 * temp * np.log(precip / 300))  # kg/m²/yr
            else:
                npp = 0.05  # Minimal NPP in harsh conditions
            
            # Calculate components
            _, e_bio = calculate_e_bio(npp)
            _, e_ppt = calculate_e_ppt(precip * 0.3, temp)  # Assume 30% effective precipitation
            
            E_BIO_grid[i, j] = e_bio
            E_PPT_grid[i, j] = e_ppt
            EEMT_grid[i, j] = e_bio + e_ppt
    
    # Create comprehensive visualization
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=['Total EEMT', 'E_BIO Component', 'E_PPT Component', 'Dominant Component'],
        specs=[[{"type": "heatmap"}, {"type": "heatmap"}],
               [{"type": "heatmap"}, {"type": "heatmap"}]]
    )
    
    # Plot 1: Total EEMT
    fig.add_trace(
        go.Heatmap(
            z=EEMT_grid,
            x=temperature,
            y=precipitation,
            colorscale='Viridis',
            name='EEMT',
            showscale=True,
            colorbar=dict(title="EEMT [MJ/m²/yr]", x=0.48)
        ),
        row=1, col=1
    )
    
    # Plot 2: E_BIO
    fig.add_trace(
        go.Heatmap(
            z=E_BIO_grid,
            x=temperature,
            y=precipitation,
            colorscale='Greens',
            name='E_BIO',
            showscale=True,
            colorbar=dict(title="E_BIO [MJ/m²/yr]", x=1.02)
        ),
        row=1, col=2
    )
    
    # Plot 3: E_PPT
    fig.add_trace(
        go.Heatmap(
            z=E_PPT_grid,
            x=temperature,
            y=precipitation,
            colorscale='Blues',
            name='E_PPT',
            showscale=True,
            colorbar=dict(title="E_PPT [MJ/m²/yr]", x=0.48, y=0.02)
        ),
        row=2, col=1
    )
    
    # Plot 4: Dominant component
    dominance = np.where(E_BIO_grid > E_PPT_grid, 1, 0)  # 1 = E_BIO, 0 = E_PPT
    fig.add_trace(
        go.Heatmap(
            z=dominance,
            x=temperature,
            y=precipitation,
            colorscale=[[0, 'blue'], [1, 'green']],
            name='Dominance',
            showscale=True,
            colorbar=dict(
                title="Dominant Component",
                x=1.02, y=0.02,
                tickvals=[0.25, 0.75],
                ticktext=['E_PPT', 'E_BIO']
            )
        ),
        row=2, col=2
    )
    
    # Add 70 MJ/m²/yr contour line to total EEMT plot
    fig.add_trace(
        go.Contour(
            z=EEMT_grid,
            x=temperature,
            y=precipitation,
            contours=dict(
                start=70, end=70, size=1,
                coloring='lines',
                showlabels=True
            ),
            line=dict(color='red', width=3),
            showscale=False,
            name='70 MJ/m²/yr'
        ),
        row=1, col=1
    )
    
    fig.update_layout(
        title="EEMT Patterns Across Climate Gradients",
        width=1200, height=800
    )
    
    # Update axis labels
    fig.update_xaxes(title_text="Temperature [°C]")
    fig.update_yaxes(title_text="Precipitation [mm/yr]")
    
    fig.show()
    
    # Calculate some interesting statistics
    threshold_mask = EEMT_grid >= 70
    below_threshold = np.sum(~threshold_mask) / (n_points * n_points) * 100
    above_threshold = np.sum(threshold_mask) / (n_points * n_points) * 100
    
    print(f"📊 Climate Gradient Analysis Results:")
    print(f"   • {below_threshold:.1f}% of climate space below 70 MJ/m²/yr threshold")
    print(f"   • {above_threshold:.1f}% of climate space above 70 MJ/m²/yr threshold")
    print(f"   • Maximum EEMT: {np.max(EEMT_grid):.1f} MJ/m²/yr")
    print(f"   • Minimum EEMT: {np.min(EEMT_grid):.1f} MJ/m²/yr")

create_climate_gradient_analysis()

## 7. Power Law Relationships

### 7.1 EEMT-Biomass Scaling

One of the most important discoveries in EEMT research is the power law relationship between EEMT and ecosystem properties:

In [ ]:
# Generate synthetic data following observed power law relationships
np.random.seed(42)

# EEMT values from literature (Rasmussen et al.)
eemt_literature = np.array([5, 12, 23, 45, 67, 89, 134, 178, 245, 312, 456, 678, 890])

# Biomass follows: Biomass = α × EEMT^β
alpha = 0.032  # kg/m²/yr per (MJ/m²/yr)^β
beta = 3.22    # Scaling exponent

# Calculate theoretical biomass
biomass_theoretical = alpha * eemt_literature**beta

# Add realistic noise
noise_factor = 0.3
biomass_observed = biomass_theoretical * (1 + np.random.normal(0, noise_factor, len(eemt_literature)))
biomass_observed = np.maximum(biomass_observed, 0.1)  # Minimum biomass

# Fit power law to "observed" data
def power_law(x, a, b):
    return a * x**b

# Log-transform for linear fitting
log_eemt = np.log10(eemt_literature)
log_biomass = np.log10(biomass_observed)

# Linear fit in log space
slope, intercept, r_value, p_value, std_err = stats.linregress(log_eemt, log_biomass)

# Convert back to power law parameters
alpha_fitted = 10**intercept
beta_fitted = slope

print(f"📈 Power Law Fit Results:")
print(f"   Biomass = {alpha_fitted:.3f} × EEMT^{beta_fitted:.2f}")
print(f"   R² = {r_value**2:.3f}")
print(f"   p-value = {p_value:.2e}")
print(f"   Theoretical: α = {alpha:.3f}, β = {beta:.2f}")

# Create power law visualization
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Linear Scale', 'Log-Log Scale'],
    specs=[[{"secondary_y": False}, {"secondary_y": False}]]
)

# Generate smooth curve for plotting
eemt_smooth = np.logspace(0, 3, 100)
biomass_smooth = alpha_fitted * eemt_smooth**beta_fitted

# Plot 1: Linear scale
fig.add_trace(
    go.Scatter(
        x=eemt_literature, y=biomass_observed,
        mode='markers',
        name='Observed Data',
        marker=dict(size=8, color='red', symbol='circle')
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=eemt_smooth, y=biomass_smooth,
        mode='lines',
        name=f'Power Law Fit (β={beta_fitted:.2f})',
        line=dict(color='blue', width=3)
    ),
    row=1, col=1
)

# Plot 2: Log-log scale
fig.add_trace(
    go.Scatter(
        x=eemt_literature, y=biomass_observed,
        mode='markers',
        name='Observed Data (log)',
        marker=dict(size=8, color='red', symbol='circle'),
        showlegend=False
    ),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(
        x=eemt_smooth, y=biomass_smooth,
        mode='lines',
        name='Power Law Fit (log)',
        line=dict(color='blue', width=3),
        showlegend=False
    ),
    row=1, col=2
)

# Update axes
fig.update_xaxes(title_text="EEMT [MJ m⁻² yr⁻¹]", row=1, col=1)
fig.update_yaxes(title_text="Biomass [kg m⁻²]", row=1, col=1)

fig.update_xaxes(title_text="EEMT [MJ m⁻² yr⁻¹]", type="log", row=1, col=2)
fig.update_yaxes(title_text="Biomass [kg m⁻²]", type="log", row=1, col=2)

fig.update_layout(
    title=f"EEMT-Biomass Power Law Relationship (R² = {r_value**2:.3f})",
    width=1000, height=450
)

fig.show()

# Add ecosystem annotations
ecosystem_examples = [
    {'name': 'Desert Scrub', 'eemt': 12, 'biomass': alpha_fitted * 12**beta_fitted},
    {'name': 'Grassland', 'eemt': 67, 'biomass': alpha_fitted * 67**beta_fitted},
    {'name': 'Deciduous Forest', 'eemt': 178, 'biomass': alpha_fitted * 178**beta_fitted},
    {'name': 'Tropical Forest', 'eemt': 456, 'biomass': alpha_fitted * 456**beta_fitted}
]

print("\n🌿 Ecosystem Predictions from Power Law:")
for eco in ecosystem_examples:
    print(f"   {eco['name']:15}: EEMT = {eco['eemt']:3d} MJ/m²/yr → Biomass = {eco['biomass']:.1f} kg/m²")

## 8. Summary and Key Takeaways

### 8.1 Fundamental Principles

This notebook has demonstrated the key thermodynamic principles underlying EEMT:

In [ ]:
# Create summary visualization
fig = go.Figure()

# Add summary text boxes
summary_points = [
    {
        'text': '<b>1. Energy Conservation</b><br>EEMT = E_BIO + E_PPT<br>Represents energy effectively<br>transferred to Critical Zone processes',
        'x': 0.15, 'y': 0.85,
        'bgcolor': 'lightgreen',
        'bordercolor': 'green'
    },
    {
        'text': '<b>2. Threshold Behavior</b><br>~70 MJ/m²/yr critical transition<br>Below: Water-limited (E_BIO dominant)<br>Above: Energy-limited (E_PPT dominant)',
        'x': 0.85, 'y': 0.85,
        'bgcolor': 'lightblue',
        'bordercolor': 'blue'
    },
    {
        'text': '<b>3. Power Law Scaling</b><br>Biomass ∝ EEMT^3.22<br>Strong nonlinear relationships<br>Emergent ecosystem organization',
        'x': 0.15, 'y': 0.15,
        'bgcolor': 'lightyellow',
        'bordercolor': 'orange'
    },
    {
        'text': '<b>4. Climate Controls</b><br>Temperature affects E_PPT<br>Precipitation controls water flux<br>Combined effects determine EEMT magnitude',
        'x': 0.85, 'y': 0.15,
        'bgcolor': 'lightcoral',
        'bordercolor': 'red'
    }
]

for point in summary_points:
    fig.add_annotation(
        x=point['x'], y=point['y'],
        text=point['text'],
        showarrow=False,
        font=dict(size=12),
        align="center",
        bordercolor=point['bordercolor'],
        borderwidth=2,
        bgcolor=point['bgcolor'],
        borderpad=10
    )

# Central equation
fig.add_annotation(
    x=0.5, y=0.5,
    text="<b>EEMT = E_BIO + E_PPT</b><br><br>Effective Energy and<br>Mass Transfer Framework",
    showarrow=False,
    font=dict(size=16, color='white'),
    align="center",
    bordercolor='black',
    borderwidth=3,
    bgcolor='darkblue',
    borderpad=15
)

fig.update_layout(
    title="<b>Critical Zone Thermodynamics: Key Concepts</b>",
    xaxis=dict(range=[0, 1], showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(range=[0, 1], showgrid=False, zeroline=False, showticklabels=False),
    width=1000, height=600,
    showlegend=False
)

fig.show()

print("\n🎯 Learning Objectives Achieved:")
print("   ✅ Understood thermodynamic principles of Critical Zone")
print("   ✅ Calculated energy balance components")
print("   ✅ Explored threshold behaviors and transitions")
print("   ✅ Visualized energy flux patterns across gradients")

print("\n📚 Next Steps:")
print("   → 02_solar_topography.ipynb: Solar radiation and terrain effects")
print("   → 03_eemt_equations.ipynb: Detailed mathematical framework")
print("   → ../02_data_sources/: Learn to access real climate and terrain data")

## 9. Exercises and Extensions

### 9.1 Practice Problems

1. **Calculate EEMT** for your local climate:
   - Look up annual precipitation and temperature for your area
   - Estimate NPP based on local vegetation
   - Calculate E_BIO and E_PPT components

2. **Threshold Analysis**: 
   - Find the precipitation-temperature combination that gives exactly 70 MJ/m²/yr
   - Compare this to major climate zones (Köppen classification)

3. **Power Law Exploration**:
   - Use the fitted power law to predict biomass for different ecosystems
   - Compare predictions to measured biomass values from literature

### 9.2 Advanced Explorations

1. **Seasonal Variation**: 
   - Modify calculations to include seasonal temperature and precipitation cycles
   - Explore how seasonal timing affects annual EEMT

2. **Uncertainty Analysis**:
   - Propagate uncertainties in temperature and precipitation through EEMT calculations
   - Assess how measurement errors affect threshold classification

3. **Literature Validation**:
   - Find published EEMT values and compare to your calculations
   - Explore how different NPP estimation methods affect results

### 9.3 Real-World Applications

1. **Climate Change Scenarios**: 
   - Project future EEMT under different warming scenarios
   - Identify regions likely to cross the 70 MJ/m²/yr threshold

2. **Ecosystem Management**:
   - Use EEMT to predict ecosystem responses to land use change
   - Design restoration strategies based on energy balance principles

3. **Research Questions**:
   - Develop hypotheses about EEMT controls in different environments
   - Design field studies to test EEMT predictions